# Pose Estimation (Camera)
This notebook presents the demonstration of pose estimation with ICP of a book for the LibreBot by T2325.

In [1]:
import numpy as np
from pydrake.all import (AbstractValue, AngleAxis, Concatenate, DiagramBuilder,
                         LeafSystem, MeshcatVisualizer, PiecewisePolynomial,
                         PiecewisePose, PointCloud, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat)

from manipulation import running_as_notebook
from manipulation.icp import IterativeClosestPoint
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.mustard_depth_camera_example import MustardPointCloud
from manipulation.pick import (MakeGripperCommandTrajectory, MakeGripperFrames,
                               MakeGripperPoseTrajectory)
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)
                                    
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pydot
import CrackerBoxPointCloud
from IPython.display import SVG, clear_output, display
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis,
                         DiagramBuilder, FindResourceOrThrow, Integrator,
                         JacobianWrtVariable, LeafSystem, MathematicalProgram,
                         MeshcatVisualizer, SnoptSolver, Solve, eq, ge, le,
                         MultibodyPlant, MultibodyPositionToGeometryPose,
                         Parser, PiecewisePolynomial, PiecewisePose,
                         Quaternion, Rgba, RigidTransform, RotationMatrix,
                         SceneGraph, Simulator, StartMeshcat, TrajectorySource)
from pydrake.all import (AbstractValue, AngleAxis, Concatenate, DiagramBuilder,
                         LeafSystem, MeshcatVisualizer, PiecewisePolynomial,
                         PiecewisePose, PointCloud, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat)

from manipulation import running_as_notebook, FindResource
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation, AddMultibodyTriad,
                                    AddShape)

from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus)
from pydrake.manipulation.planner import DoDifferentialInverseKinematics
from manipulation.icp import IterativeClosestPoint
from pydrake.all import (Box, Sphere, PlanarJoint, RevoluteJoint)
from datetime import datetime

if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
print(str(datetime.now()))
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://29fb9f43-5ddf-4394-96be-7b378ea2c223.deepnoteproject.com/7001/
2023-04-04 21:52:53.084339


In [3]:
model_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf    
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0, -0.1, 0.0]
        rotation: !Rpy { deg: [0, 0, 0]}
        
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, -0., 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

- add_frame:
    name: bin0_origin
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 70.0, 90.0 ]}
      translation: [-0.1, -1.1, 0.45]

- add_model:
    name: bin0
    file: package://drake/examples/manipulation_station/models/bin.sdf

- add_weld:
    parent: bin0_origin
    child: bin0::bin_base

- add_frame:
    name: shelf1_goal
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0, 180.0 ]}
      translation: [0.95, 0.05, 0.48]

- add_model:
    name: shelf1
    file: package://manipulation/shelves.sdf

- add_weld:
    parent: shelf1_goal
    child: shelf1::shelves_body


- add_model:
    name: floor
    file: package://manipulation/floor.sdf

- add_weld:
    parent: world
    child: floor::box
    X_PC:
        translation: [0, 0, -.5]

# cracker box
# add cracker box
- add_model:
    name: cracker_box
    file: package://drake/manipulation/models/ycb/sdf/003_cracker_box.sdf
    default_free_body_pose:
        base_link_cracker:
            translation: [-0.1, -0.95, 0.45]
            rotation: !Rpy { deg: [-90, 0, 90] }

# add cameras
- add_frame:
    name: camera3_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [80., 150, -40]}
        translation: [0.25, -.5, .5]

- add_model:
    name: camera3
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera3_origin
    child: camera3::base

- add_frame:
    name: camera4_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [50, 180, 0]}
        translation: [-0.05, -.4, 1]

- add_model:
    name: camera4
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera4_origin
    child: camera4::base

- add_frame:
    name: camera5_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [80., 210, 50]}
        translation: [-0.5, -.5, .5]

- add_model:
    name: camera5
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera5_origin
    child: camera5::base

""" 

# Putting it all together

In the code above, we worked with a point cloud using functions.  To assemble this into a full-stack manipulation system, we need to specify the timing semantics of when those functions are called.  That's precisely what Drake's systems framework provides.  I've introduced two systems below: 
- `MustardIterativeClosestPoint` system that takes the camera inputs and outputs the pose estimate using ICP, and 
- `PickAndPlaceTrajectory` system that takes this pose estimate (and the state of the robot), computes the trajectory, and stores that trajectory in its Context so that it can output the instantaneous command.  

We don't use a `TrajectorySource` here, because the trajectory is not known when we first build the Diagram... the information we need to plan the trajectory requires reading sensors at runtime.

In [4]:
def OurMakeGripperFrames(X_G, t0,tmpReal):
    """
    Takes a partial specification with X_G["initial"], X_G["pick"], and
    X_G["place"], and returns a X_G and times with all of the pick and place
    frames populated.
    """
    # pregrasp is negative y in the gripper frame (see the figure!).
    X_GgraspGpregrasp = RigidTransform([0, -0.2, 0])

    X_G["prepick"] = X_G["pick"] @ X_GgraspGpregrasp
    X_G["preplace"] = X_G["place"] @ X_GgraspGpregrasp

    # I'll interpolate a halfway orientation by converting to axis angle and
    # halving the angle.
    X_GinitialGprepick = X_G["initial"].inverse() @ X_G["prepick"]
    angle_axis = X_GinitialGprepick.rotation().ToAngleAxis()
    X_GinitialGprepare = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        X_GinitialGprepick.translation() / 2.0)
    X_G["prepare"] = X_G["initial"] @ X_GinitialGprepare
    p_G = np.array(X_G["prepare"].translation())
    p_G[2] = 0.5
    # To avoid hitting the cameras, make sure the point satisfies x - y < .5
    if p_G[0] - p_G[1] < .5:
        scale = .5 / (p_G[0] - p_G[1])
        p_G[:1] /= scale
    X_G["prepare"].set_translation(p_G)

    X_GprepickGpreplace = X_G["prepick"].inverse() @ X_G["preplace"]
    angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
    X_GprepickGclearance = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        X_GprepickGpreplace.translation() / 2.0)
    X_G["clearance"] = tmpReal
    p_G = np.array(X_G["clearance"].translation())
    p_G[2] = 0.5
    # To avoid hitting the cameras, make sure the point satisfies x - y < .5
    if p_G[0] - p_G[1] < .5:
        scale = .5 / (p_G[0] - p_G[1])
        p_G[:1] /= scale
    X_G["clearance"].set_translation(p_G)

    # Now let's set the timing
    times = {"initial": t0}
    prepare_time = 10.0 * np.linalg.norm(X_GinitialGprepare.translation())
    times["prepare"] = times["initial"] + prepare_time
    times["prepick"] = times["prepare"] + prepare_time
    
    # Allow some time for the gripper to close.
    times["pick_start"] = times["prepick"] + 2.0
    times["pick_end"] = times["pick_start"] + 2.0
    X_G["pick_start"] = X_G["pick"]
    X_G["pick_end"] = X_G["pick"]
    times["postpick"] = times["pick_end"] + 2.0
    X_G["postpick"] = X_G["prepick"]
    time_to_from_clearance = 10.0 * np.linalg.norm(
        X_GprepickGclearance.translation())
    times["clearance"] = times["postpick"] + time_to_from_clearance
    times["preplace"] = times["clearance"] + time_to_from_clearance
    times["place_start"] = times["preplace"] + 2.0
    times["place_end"] = times["place_start"] + 2.0
    times["postplace"] = times["place_end"] + 2.0
    
    #X_G["preplace"].set_translation( X_G["preplace"].translation() - [-0.05, 0,0.35] )
    X_G["place_start"] = X_G["place"]
    X_G["place_end"] = RigidTransform(X_G["place"])
   # X_G["place_end"].set_translation( X_G["place_end"].translation() - [-0.05, 0,0.0] )
    # X_G["postplace"] = X_G["preplace"]
    X_G["postplace"] = RigidTransform( [0, 0.5, 0.45] )
    return X_G, times



In [5]:
# Takes 3 point clouds (in world coordinates) as input, and outputs and estimated pose for the mustard bottle.
class MustardIterativeClosestPoint(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        model_point_cloud = AbstractValue.Make(PointCloud(0))
        self.DeclareAbstractInputPort("cloud0", model_point_cloud)
        self.DeclareAbstractInputPort("cloud1", model_point_cloud)
        self.DeclareAbstractInputPort("cloud2", model_point_cloud)

        self.DeclareAbstractOutputPort(
            "X_WO", lambda: AbstractValue.Make(RigidTransform()),
            self.EstimatePose)

        self.mustard = CrackerBoxPointCloud.CrackerBoxPointCloud()
        meshcat.SetObject("icp_scene", self.mustard)
        meshcat.SetObject("icp_model", self.mustard)


    def EstimatePose(self, context, output):
        pcd = []
        for i in range(3):
            cloud = self.get_input_port(i).Eval(context)
            # print input clouds
            # print("cloud", i, cloud.xyzs())
            
            # plot point clouds


            # TODO: create image from xyzs
            # # Plot the two images.
            # plt.subplot(121)
            # plt.imshow(cloud)
            # plt.title('cloud', i)
            # plt.subplot(122)
            # plt.imshow(np.squeeze(depth_image.data))
            # plt.title('Depth image')
            # #mpld3.display()
            # plt.show()

#                 cloud.Crop(lower_xyz=[-0.5, -1, 0.5], upper_xyz=[0.8, 0.8, 0.8]))
#                 cloud.Crop(lower_xyz=[-0.5, -0.87, 0.5], upper_xyz=[0.8, 0.8, 0.6]))
#               cloud.Crop(lower_xyz=[-0.5, -0.87, 0.4455], upper_xyz=[0.8, 0.8, 0.6]))

            pcd.append(
                #cloud.Crop(lower_xyz=[.4, -.2, 0.001], upper_xyz=[.6, .3, .3]))
                #
                # cloud.Crop(lower_xyz=[-0.11, -0.96, 0.43], upper_xyz=[-0., -0.94, 0.46]))
                #cloud.Crop(lower_xyz=[-0.5, -0.87, 0.2], upper_xyz=[0.8, 0.8, 0.6]))
                #cloud.Crop(lower_xyz=[-0.5, -0.87, 0.45], upper_xyz=[0.1, -0.86, 0.6]))
                cloud.Crop(lower_xyz=[-0.17, -1.07, -2], upper_xyz=[0, 0, 0.6]))


        merged_pcd = Concatenate(pcd)
        down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
        meshcat.SetObject("icp_observations",
                          down_sampled_pcd,
                          point_size=0.001)

        # takes icp_observations + model cloud , returns icp?s
        X_WOhat, chat = IterativeClosestPoint(
            self.mustard.xyzs(),
            down_sampled_pcd.xyzs(),
            meshcat=meshcat,
            meshcat_scene_path="icp_scene")
        print("X_WOhat = {}".format(X_WOhat))
        output.set_value(X_WOhat)

In [6]:
# set timestamps
time = []
scale = 100
for i in range(5):
    time.append(i * scale)

# set the corresponding locations with timestamps
FB_traj_X_G = PiecewisePose.MakeLinear(
    time,
    [
        RigidTransform(
            RotationMatrix.MakeZRotation(0),
            p=[0, 0, 0.0],
        ),
        RigidTransform(
            RotationMatrix.MakeZRotation(0),
            p=[0, 0, 0.0],
        ),
        RigidTransform(
            RotationMatrix.MakeZRotation(0),
            p=[0, 0, 0.0],
        ),
        RigidTransform(
            RotationMatrix.MakeZRotation(0),
            p=[0, 0, 0.0],
        ),
        RigidTransform(
            RotationMatrix.MakeZRotation(0),
            p=[0, 0, 0.0],
        ),
        # RigidTransform(
        #     RotationMatrix.MakeZRotation(-np.pi / 2),
        #     p=[-scale, -scale, 0.0],
        # ),
        # RigidTransform(
        #     RotationMatrix.MakeZRotation(-np.pi),
        #     p=[-scale, scale, 0.0],
        # ),
        # RigidTransform(
        #     RotationMatrix.MakeZRotation(-np.pi + np.pi / 2),
        #     p=[scale, scale, 0.0],
        # ),
        # RigidTransform(
        #     RotationMatrix.MakeZRotation(-2 * np.pi),
        #     p=[scale, -scale, 0.0],
        # ),
        # RigidTransform(
        #     RotationMatrix.MakeZRotation(-np.pi / 2),
        #     p=[-scale, -scale, 0.0],
        # ),
    ],
)

FB_traj_p_G = FB_traj_X_G.get_position_trajectory()
p_G = FB_traj_p_G.vector_values(FB_traj_p_G.get_segment_times())
plt.plot(FB_traj_p_G.get_segment_times(), p_G.T)
plt.legend(["x", "y", "z"])
plt.title("p_G")
# mpld3.display()

FB_traj_R_G = FB_traj_X_G.get_orientation_trajectory()
FB_R_G = FB_traj_R_G.vector_values(FB_traj_R_G.get_segment_times())
plt.plot(FB_traj_R_G.get_segment_times(), FB_R_G.T)
plt.legend(["qx", "qy", "qz", "qw"])
plt.title("R_G")
mpld3.display()

In [7]:
class FreeBoxTrajectoriesToPosition(LeafSystem):
    def __init__(self, plant, traj_p_G, traj_R_G):
        LeafSystem.__init__(self)
        self.plant = plant
        self.freebox_body = plant.GetBodyByName("freebox")
        self.freebox_joint = plant.GetJointByName("freebox_joint")
        self.traj_p_G = traj_p_G
        self.traj_R_G = traj_R_G
        self.plant_context = plant.CreateDefaultContext()

        self.DeclareVectorOutputPort(
            "position", plant.num_positions(), self.CalcPositionOutput
        )
    def CalcPositionOutput(self, context, output):
        t = context.get_time()
        X_G = RigidTransform(Quaternion(self.traj_R_G.value(t)), self.traj_p_G.value(t))
        # print(X_G.translation()[:2])
        # self.plant.SetFreeBodyPose(self.plant_context, self.freebox_body, X_G)
        self.freebox_joint.set_translation(self.plant_context, X_G.translation()[:2])
        self.freebox_joint.set_rotation(self.plant_context, X_G.rotation().ToRollPitchYaw().yaw_angle())
        output.SetFromVector(self.plant.GetPositions(self.plant_context))

In [8]:
class PickAndPlaceTrajectory(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._gripper_body_index = plant.GetBodyByName("body").index()
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()]))
        self.DeclareAbstractInputPort("X_WO",
                                      AbstractValue.Make(RigidTransform()))

        self.DeclareInitializationUnrestrictedUpdateEvent(self.Plan)
        self._traj_X_G_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose()))
        self._traj_wsg_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial()))

        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()),
            self.CalcGripperPose)
        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

    def Plan(self, context, state):
        X_G = {
            "initial":
                self.get_input_port(0).Eval(context)
                [int(self._gripper_body_index)]
        }

        X_O = {
            "initial": self.get_input_port(1).Eval(context),
            "goal": RigidTransform( [1.15, 0.0, 0.45])
            #"goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2.0), [1.15, 0.0, 0.0])
        }
        
      #  rotation: !Rpy { deg: [0.0, 70.0, 180.0 ]}
      #  translation: [1.2, 0.05, 0.45]
        X_GgraspO = RigidTransform( RotationMatrix.MakeXRotation(np.pi)  ,[0.03,0.03,0.06])
        X_OGgrasp = X_GgraspO.inverse()
        X_G["pick"] = X_O["initial"] @ X_OGgrasp

        tmp = X_O["goal"] @ X_OGgrasp


            #tmp = X_O["goal"] @ X_OGgrasp
        # I'll interpolate a halfway orientation by converting to axis angle and halving the angle.
        X_GprepickGpreplace = X_G["pick"].inverse() @ tmp
        angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
        X_GprepickGclearance = RigidTransform(
            AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
            X_GprepickGpreplace.translation() / 2.0 + np.array([0.8, -1, 2]),
        )
        
        # the arm should rotate to avoid collision, the Z rotation is for that!
        X_GprepickGclearance.set_rotation(X_GprepickGclearance.rotation() @ RotationMatrix.MakeZRotation(-np.pi / 2.0))
        tmpReal = X_G["pick"] @ X_GprepickGclearance
        X_G["place"] = X_O["goal"] @ X_OGgrasp


        if True:  # Useful for debugging
            AddMeshcatTriad(meshcat, "X_Oinitial", X_PT=X_O["initial"])
            AddMeshcatTriad(meshcat, "X_Gpick", X_PT=X_G["pick"])
            AddMeshcatTriad(meshcat, "X_Gplace", X_PT=X_G["place"]) 
            AddMeshcatTriad(meshcat, "X_goal", X_PT=X_O["goal"])
            AddMeshcatTriad(meshcat, f"gripper", X_PT=X_G["initial"])


        X_G, times = OurMakeGripperFrames(X_G,0,tmpReal)
        print(f"Planned {times['postplace']} second trajectory.")


        traj_X_G = MakeGripperPoseTrajectory(X_G, times)
        traj_wsg_command = MakeGripperCommandTrajectory(times)

        state.get_mutable_abstract_state(int(
            self._traj_X_G_index)).set_value(traj_X_G)
        state.get_mutable_abstract_state(int(
            self._traj_wsg_index)).set_value(traj_wsg_command)

    def start_time(self, context):
        return context.get_abstract_state(
            int(self._traj_X_G_index)).get_value().start_time()

    def end_time(self, context):
        return context.get_abstract_state(
            int(self._traj_X_G_index)).get_value().end_time()

    def CalcGripperPose(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.set_value(context.get_abstract_state(int(
            self._traj_X_G_index)).get_value().GetPose(context.get_time()))

    def CalcWsgPosition(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.SetFromVector(
            context.get_abstract_state(int(
                self._traj_wsg_index)).get_value().value(context.get_time()))

In [9]:
def icp_pick_and_place_demo():
    builder = DiagramBuilder()
    # scene_graph = builder.AddSystem(SceneGraph())
    
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=0.002))
    
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")

    # Find camera frames
    frames = []
    for i in range(3):
        frames.append(plant.GetFrameByName(f"camera{i+3}_origin"))
        print(f"camera{i+3}_origin, {frames[i]}")
    # for each frame add triads
    # for i in range(1):
    #     AddMultibodyTriad(frames[i], scene_graph, length=.1, radius=0.005)

    # Add meshcet triads for cameras

    # for i in range(3):
    #     AddMeshcatTriad(meshcat, f"camera{i+3}_origin", X_PT=frames[i].GetFixedPoseInBodyFrame())



    icp = builder.AddSystem(MustardIterativeClosestPoint())
    builder.Connect(station.GetOutputPort("camera3_point_cloud"),
                    icp.get_input_port(0))
    builder.Connect(station.GetOutputPort("camera4_point_cloud"),
                    icp.get_input_port(1))
    builder.Connect(station.GetOutputPort("camera5_point_cloud"),
                    icp.get_input_port(2))

    

    plan = builder.AddSystem(PickAndPlaceTrajectory(plant))
    builder.Connect(station.GetOutputPort("body_poses"),
                    plan.GetInputPort("body_poses"))
    builder.Connect(icp.GetOutputPort("X_WO"), plan.GetInputPort("X_WO"))

    robot = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Set up differential inverse kinematics.
    diff_ik = AddIiwaDifferentialIK(builder, robot)
    builder.Connect(diff_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(plan.GetOutputPort("X_WG"),
                    diff_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    diff_ik.GetInputPort("robot_state"))
    builder.Connect(plan.GetOutputPort("wsg_position"),
                    station.GetInputPort("wsg_position"))


    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    diagram = builder.Build()

    simulator = Simulator(diagram)
    context = simulator.get_context()

    simulator.Initialize()

    # color_image = station.GetOutputPort("camera3_point_cloud").Eval(context)
    # depth_image = station.GetOutputPort("camera3_point_cloud").Eval(context)

    # # Plot the two images.
    # plt.subplot(121)
    # plt.imshow(color_image.data)
    # plt.title('Color image')
    # plt.subplot(122)
    # plt.imshow(np.squeeze(depth_image.data))
    # plt.title('Depth image')
    # #mpld3.display()
    # plt.show()

    if True: # draw the trajectory triads
        X_G_traj = plan.GetMyContextFromRoot(context).get_abstract_state(
            0).get_value()
        for t in np.linspace(X_G_traj.start_time(), X_G_traj.end_time(), 40):
            AddMeshcatTriad(meshcat,
                            f"X_G/({t})",
                            X_PT=X_G_traj.GetPose(t),
                            length=.1,
                            radius=0.004)

    if running_as_notebook:
        visualizer.StartRecording(False)
        simulator.AdvanceTo(plan.end_time(plan.GetMyContextFromRoot(context)))
        visualizer.PublishRecording()
    else:
        simulator.AdvanceTo(0.1)

print(str(datetime.now()))
icp_pick_and_place_demo()

2023-04-04 21:52:53.350436
camera3_origin, <FixedOffsetFrame_[float] name='camera3_origin' index=46 model_instance=0>
camera4_origin, <FixedOffsetFrame_[float] name='camera4_origin' index=49 model_instance=0>
camera5_origin, <FixedOffsetFrame_[float] name='camera5_origin' index=52 model_instance=0>
X_WOhat = RigidTransform(
  R=RotationMatrix([
    [0.9990687370300293, -0.008659192360937595, -0.04226946085691452],
    [0.00529705872759223, 0.9968594312667847, -0.07901392132043839],
    [0.04282090812921524, 0.07871641963720322, 0.995976984500885],
  ]),
  p=[-0.0883515477180481, -0.9427328109741211, 0.3291110396385193],
)
Planned 31.035537517547365 second trajectory.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7f664b82-843d-4a5a-bb36-dfd56c0fa9be' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>